# build metadata for tgt files

In [1]:
import pandas as pd
from functions import load_txt_as_lst

In [2]:
src_meta_path = "/scratch/craig.car/src_data/urn:cts:greekLit:tlg0001.tlg001.json"
src_meta = pd.read_json(src_meta_path)
src_meta

,loc,start,length
0,urn:cts:greekLit:tlg0001.tlg001:1.1,0,45
1,urn:cts:greekLit:tlg0001.tlg001:1.2,45,47
2,urn:cts:greekLit:tlg0001.tlg001:1.3,92,34
3,urn:cts:greekLit:tlg0001.tlg001:1.4,126,40
4,urn:cts:greekLit:tlg0001.tlg001:1.5,166,45
...,...,...,...
5829,urn:cts:greekLit:tlg0001.tlg001:4.1777,254548,43
5830,urn:cts:greekLit:tlg0001.tlg001:4.1778,254591,43
5831,urn:cts:greekLit:tlg0001.tlg001:4.1779,254634,43
5832,urn:cts:greekLit:tlg0001.tlg001:4.1780,254677,42


In [3]:
test_output_xml = "/scratch/craig.car/test.par"
test_par = load_txt_as_lst(test_output_xml)

In [4]:
test_par[1]

"book=1,chapter=1\t6\tCe que je tente aujourd’hui, plusieurs l’ont tenté avant moi; on a généralement loué leur entreprise. Chez quelques-uns même, ce n’est pas seulement au courageux effort que de bons juges ont applaudi. Où l’espoir de réussir toujours serait chimérique, ils ont eu du moins le bonheur de réussir souvent. La traduction des Choéphores, par M. Puech, que l’Université a trop tôt perdu, a eu pour elle des suffrages d’une grande autorité et est encore à bon droit estimée. M. Léon Halévy, après avoir déjà donné tant de preuves de son talent poétique, a tout récemment mérité d’être couronné par l’Académie française pour son troisième volume de la Grèce tragique, qui renferme une traduction des Euménides1. Sur un de nos théâtres de Paris, M. Alexandre Dumas a fait représenter une Orestie le 5 janvier 1856. Je devrais le compter, lui aussi, parmi ceux dont la rencontre était fort à craindre pour moi sur le chemin où ma témérité m’a engagé, s’il n’avait imité la célèbre trilogi

In [5]:
test_par[1].split("\t")

['book=1,chapter=1',
 '6',
 "Ce que je tente aujourd’hui, plusieurs l’ont tenté avant moi; on a généralement loué leur entreprise. Chez quelques-uns même, ce n’est pas seulement au courageux effort que de bons juges ont applaudi. Où l’espoir de réussir toujours serait chimérique, ils ont eu du moins le bonheur de réussir souvent. La traduction des Choéphores, par M. Puech, que l’Université a trop tôt perdu, a eu pour elle des suffrages d’une grande autorité et est encore à bon droit estimée. M. Léon Halévy, après avoir déjà donné tant de preuves de son talent poétique, a tout récemment mérité d’être couronné par l’Académie française pour son troisième volume de la Grèce tragique, qui renferme une traduction des Euménides1. Sur un de nos théâtres de Paris, M. Alexandre Dumas a fait représenter une Orestie le 5 janvier 1856. Je devrais le compter, lui aussi, parmi ceux dont la rencontre était fort à craindre pour moi sur le chemin où ma témérité m’a engagé, s’il n’avait imité la célèbre 

In [6]:
import spacy
import spacy_fastlang

In [7]:
spacy_fr_sm = spacy.load("fr_core_news_sm")
spacy_fr_sm.add_pipe("language_detector")

In [24]:
reconstructed = []
section_idx2section_name = {}
prev_meta = test_par[0].split("\t")[0]
current_txt = ""
reconstructed_idx = 0
for fragment in test_par:
    meta = fragment.split("\t")[0]
    text = fragment.split("\t")[2]
    if meta == prev_meta:
        # still in same section, check lang before adding text
        if spacy_fr_sm(text)._.language == "fr":
            current_txt += " "+text
    else:
        # started a new section, append current_txt
        reconstructed.append(current_txt)
        # add metadata to dict
        section_idx2section_name[reconstructed_idx] = meta
        # check lang then re-initialize current_txt
        if spacy_fr_sm(text)._.language == "fr":
            current_txt = text
        # re-initialize reconstructed_idx regardless of lang
        reconstructed_idx += 1
        # re-initialize prev_meta
        prev_meta = meta
# add last section to reconstructed
reconstructed.append(current_txt)
section_idx2section_name[reconstructed_idx] = meta

In [25]:
len(section_idx2section_name)

44

In [26]:
len(reconstructed)

44

In [28]:
section_idx2section_name[43]

'book=3,chapter=3,section=26'

In [29]:
reconstructed[-1]

"Souveraine Athéné, par l’ordre d’Apollon\n J’arrive : au malheureux que ton accueil soit bon!\n Ce n’est point teint de sang qu’ici je me présente :\n Sur ma main a séché la tache pâlissante;\n Tandis que s’effaçaient des souvenirs vieillis,\n J’ai d’asile en asile erré par tout pays.\n Du continent, des mers j’ai franchi l’étendue,\n Envoyé par la voix dans Delphes entendue.\n Au pied de ton image, en ton temple clément\n Je viens m’asseoir; et là j’attends le jugement.\n Bien! c’est de son passage un évident indice;\n Suis la trace de sang, muette délatrice.\n Comme le chien dépiste un faon qui fuit blessé,\n A ces gouttes je vois qu’il est par là passé.\n Un souffle haletant fatigue ma poitrine;\n Car par toute la terre à courir je m’obstine;\n Et sur mer, sans avoir les ailes d’un oiseau,\n J’ai volé, j’ai suivi son rapide vaisseau.\n Mais il est par ici caché, je le devine;\n L’odeur du sang humain a flatté ma narine.\n Cherchons, cherchons de toutes parts :\n Que rien n’échappe 

# check embeds from runner.py test runs

In [1]:
import numpy as np
import pandas as pd
from functions import load_txt_as_lst

In [2]:
prefix = "/scratch/craig.car/src_data/urn:cts:greekLit:tlg0004.tlg001"
emb_path = prefix+".emb"
dim = 768
embeds = np.fromfile(emb_path, dtype=np.float32, count=-1)
embeds.resize(embeds.shape[0] // dim, dim)
embeds.shape

(64241, 768)

In [3]:
txt_path = prefix+".overlaps"


In [4]:
txt = load_txt_as_lst(txt_path)


In [5]:
len(txt)

64241

# clean_par bug: duplicated text + incorrect meta

In [20]:
import spacy
import spacy_fastlang

In [21]:
lang_ = "fr"
spacy_model_ = spacy.load("fr_core_news_sm")
spacy_model_.add_pipe("language_detector")

In [22]:
# import fasttext
# from huggingface_hub import hf_hub_download

In [23]:
# model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")

In [24]:
# model = fasttext.load_model(model_path)

In [25]:
def clean_par(pars_lst, spacy_model, lang):
    '''
    1. Merges sections split during extraction from XML while filtering out
    text in different language (e.g. for editions including original text).
    2. Builds metadata in the form of a dict mapping the text's index to
    section name. 
    '''
    reconstructed = []
    section_idx2section_name = {}
    prev_meta = pars_lst[0].split("\t")[0]
    current_txt = ""
    reconstructed_idx = 0

    for idx, fragment in enumerate(pars_lst):
        meta = fragment.split("\t")[0]
        text = fragment.split("\t")[2]
        if meta == prev_meta:
            # still in same section, check lang before adding text
            if spacy_model(text.lower())._.language == lang:
                current_txt += " "+text
        else:
            # started a new section, append current_txt
            if current_txt != "":
                reconstructed.append(current_txt)
                # add metadata to dict
                section_idx2section_name[reconstructed_idx] = prev_meta
                # re-initialize reconstructed_idx regardless of lang
                reconstructed_idx += 1
            # re-initialize prev_meta
            prev_meta = meta
            # check lang then re-initialize current_txt
            if spacy_model(text.lower())._.language == lang:
                current_txt = text
            else:
                current_txt = ""
    # add last section to reconstructed
    reconstructed.append(current_txt)
    section_idx2section_name[reconstructed_idx] = meta
    
    return reconstructed, section_idx2section_name


In [29]:
from functions import load_txt_as_lst
file = "/scratch/craig.car/french_trans-dev/cicero_11_1826.par"
text_par = load_txt_as_lst(file)
clean_pars, idx2section_name = clean_par(text_par, spacy_model_, lang_)

In [31]:
idx2section_name

{0: 'book=5,=',
 1: 'book=6,=',
 2: 'book=7,=',
 3: 'book=1',
 4: 'book=2',
 5: 'book=3',
 6: 'book=4',
 7: 'book=5',
 8: 'book=6',
 9: 'book=7'}

In [32]:
for section in clean_pars:
    print(section[:100].replace("\n",""))

ORATIO SECUNDA ET VICESIMA.
ORATIO TERTIA ET VICESIMA.
ORATIO QUARTA ET VICESIMA.
DISCOURS DIX-HUITIÈME. I. Romains, quoique je n’ait pas coutume, lorsque je plaide une cause, de co
DISCOURS DIX-NEUVIÈME. I. Jusques à quand abuseras-tu de notre patience, Catilina? combien de temps
PRONONCÉ DEVANT LE PEUPLE. DISCOURS VINGTIÈME. I. Enfin, Romains, cet audacieux dont la fureur sa-
DISCOURS VINGT-UNIÈME. I. La république, citoyens, votre vie, vos biens, vos fortunes, vos femmes, 
DISCOURS VINGT-DEUXIÈME. I. Je vois, pères conscrits, que tons vos regards sont attachés sur moi1. 
DISCOURS VINGT-TROISIÈME. I. Romains, le jour où, suivant l’usage établi par nos ancêtres, après av
DISCOURS VINGT-QUATRIÈME. I. Mon premier vœu, Romains, eût été de voir P. Sylla se maintenir dans t


In [33]:
clean_pars[2]

'ORATIO QUARTA ET VICESIMA.\n'

In [34]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path)


# Multirow tables in pd

In [34]:
# import pandas as pd
# import numpy as np
# df = pd.DataFrame({"Text1": ["HI", "HI", "HI", "Hello", "Hello", "Hiiii", "", "", "How are You?", "Hellooo", ""],
#                    "Text2": ["welcome", "Howdy", "greetings", "good day.", "good morning.", "welcome", "Howdy", "greetings", "", "good day.", "good morning."]})
# result = df.groupby(df["Text1"].replace("", np.nan).ffill()).agg({"Text2": "/".join})
# result

In [35]:
# df['Text2'] = df[['Text1','Text2']].groupby(['Text1'])['Text2'].transform(lambda x: ','.join(x))
# df[['Text1','Text2']].drop_duplicates()
# df

In [37]:
# def pretty_print(df):
#     return display(HTML(df.to_html().replace("\\n","<br>")))

# data = [
#        {'Greek': "ὃς δ᾽ ἂν ὑμῶν παραμείνῃ, ὁρῶν ὃν τρόπον ἡμεῖς τάς τε δίκας δικάζομεν καὶ τἆλλα τὴν πόλιν διοικοῦμεν, ἤδη φαμὲν τοῦτον ὡμολογηκέναι ἔργῳ ἡμῖν ἃ ἂν ἡμεῖς κελεύωμεν ποιήσειν ταῦτα, καὶ τὸν μὴ πειθόμενον τριχῇ φαμεν ἀδικεῖν, ὅτι τε γεννηταῖς οὖσιν ἡμῖν οὐ πείθεται, καὶ ὅτι τροφεῦσι, καὶ ὅτι ὁμολογήσας ἡμῖν πείσεσθαι οὔτε πείθεται οὔτε πείθει ἡμᾶς, εἰ μὴ καλῶς τι ποιοῦμεν, προτιθέντων ἡμῶν καὶ οὐκ ἀγρίως ἐπιταττόντων ποιεῖν ἃ ἂν κελεύωμεν, ἀλλὰ ἐφιέντων δυοῖν θάτερα, ἢ πείθειν ἡμᾶς ἢ ποιεῖν, τούτων οὐδέτερα ποιεῖ.", 'Fowler': '[0] But we say that whoever of you stays here, seeing how we administer justice and how we govern the state in other respects, has thereby entered into an agreement with us to do what we command;\n[1] and we say that he who does not obey does threefold wrong, because he disobeys us who are his parents, because he disobeys us who nurtured him, and because after agreeing to obey us he neither obeys us nor convinces us that we are wrong, though we give him the opportunity and do not roughly order him to do what we command, but when we allow him a choice of two things, either to convince us of error or to do our bidding, he does neither of these things.”', 'Jowett': '[0] But he who has experience of the manner in which we order justice and administer the State, and still remains, has entered into an implied contract that he will do as we command him.\n[1] And he who disobeys us is, as we maintain, thrice wrong:\n[2] first, because in disobeying us he is disobeying his parents;\n[3] secondly, because we are the authors of his education;\n[4] thirdly, because he has made an agreement with us that he will duly obey our commands;\n[5] and he neither obeys them nor convinces us that our commands are wrong;\n[6] and we do not rudely impose them, but give him the alternative of obeying or convincing us;'},
#     {'Greek': 'ταύταις δή φαμεν καὶ σέ, ὦ Σώκρατες, ταῖς αἰτίαις ἐνέξεσθαι, εἴπερ ποιήσεις ἃ ἐπινοεῖς, καὶ οὐχ ἥκιστα Ἀθηναίων σέ, ἀλλ᾽ ἐν τοῖς μάλιστα.', 'Fowler':'[2] and we say that he who does not obey does threefold wrong, because he disobeys us who are his parents, because he disobeys us who nurtured him, and because after agreeing to obey us he neither obeys us nor convinces us that we are wrong, though we give him the opportunity and do not roughly order him to do what we command, but when we allow him a choice of two things, either to convince us of error or to do our bidding, he does neither of these things.”', 'Jowett':'[7] that is what we offer and he does neither.\n[8] These are the sort of accusations to which, as we were saying, you, Socrates, will be exposed if you accomplish your intentions;\n[9] you, above all other Athenians."'}
#        ]

# pretty_print(df)

In [41]:
import pandas as pd
from IPython.display import display, HTML

'''
https://stackoverflow.com/questions/40990700/pandas-dataframes-in-jupyter-columns-of-equal-width-and-centered
https://stackoverflow.com/questions/34322448/pretty-printing-newlines-inside-a-string-in-a-pandas-dataframe
'''

data = [
       {'Greek': "ὃς δ᾽ ἂν ὑμῶν παραμείνῃ, ὁρῶν ὃν τρόπον ἡμεῖς τάς τε δίκας δικάζομεν καὶ τἆλλα τὴν πόλιν διοικοῦμεν, ἤδη φαμὲν τοῦτον ὡμολογηκέναι ἔργῳ ἡμῖν ἃ ἂν ἡμεῖς κελεύωμεν ποιήσειν ταῦτα, καὶ τὸν μὴ πειθόμενον τριχῇ φαμεν ἀδικεῖν, ὅτι τε γεννηταῖς οὖσιν ἡμῖν οὐ πείθεται, καὶ ὅτι τροφεῦσι, καὶ ὅτι ὁμολογήσας ἡμῖν πείσεσθαι οὔτε πείθεται οὔτε πείθει ἡμᾶς, εἰ μὴ καλῶς τι ποιοῦμεν, προτιθέντων ἡμῶν καὶ οὐκ ἀγρίως ἐπιταττόντων ποιεῖν ἃ ἂν κελεύωμεν, ἀλλὰ ἐφιέντων δυοῖν θάτερα, ἢ πείθειν ἡμᾶς ἢ ποιεῖν, τούτων οὐδέτερα ποιεῖ.", 'Fowler': '[0] But we say that whoever of you stays here, seeing how we administer justice and how we govern the state in other respects, has thereby entered into an agreement with us to do what we command;\n[1] and we say that he who does not obey does threefold wrong, because he disobeys us who are his parents, because he disobeys us who nurtured him, and because after agreeing to obey us he neither obeys us nor convinces us that we are wrong, though we give him the opportunity and do not roughly order him to do what we command, but when we allow him a choice of two things, either to convince us of error or to do our bidding, he does neither of these things.”', 'Jowett': '[0] But he who has experience of the manner in which we order justice and administer the State, and still remains, has entered into an implied contract that he will do as we command him.\n[1] And he who disobeys us is, as we maintain, thrice wrong:\n[2] first, because in disobeying us he is disobeying his parents;\n[3] secondly, because we are the authors of his education;\n[4] thirdly, because he has made an agreement with us that he will duly obey our commands;\n[5] and he neither obeys them nor convinces us that our commands are wrong;\n[6] and we do not rudely impose them, but give him the alternative of obeying or convincing us;'},
    {'Greek': 'ταύταις δή φαμεν καὶ σέ, ὦ Σώκρατες, ταῖς αἰτίαις ἐνέξεσθαι, εἴπερ ποιήσεις ἃ ἐπινοεῖς, καὶ οὐχ ἥκιστα Ἀθηναίων σέ, ἀλλ᾽ ἐν τοῖς μάλιστα.', 'Fowler':'[2] and we say that he who does not obey does threefold wrong, because he disobeys us who are his parents, because he disobeys us who nurtured him, and because after agreeing to obey us he neither obeys us nor convinces us that we are wrong, though we give him the opportunity and do not roughly order him to do what we command, but when we allow him a choice of two things, either to convince us of error or to do our bidding, he does neither of these things.”', 'Jowett':'[7] that is what we offer and he does neither.\n[8] These are the sort of accusations to which, as we were saying, you, Socrates, will be exposed if you accomplish your intentions;\n[9] you, above all other Athenians."'}
       ]
df = pd.DataFrame(data)

d = dict(selector="th",
    props=[('text-align', 'center')])
df.style.set_properties(**{'width':'30em', 'text-align':'left', 'white-space': 'pre-wrap'})\
        .set_table_styles([d])

,Greek,Fowler,Jowett
0,"ὃς δ᾽ ἂν ὑμῶν παραμείνῃ, ὁρῶν ὃν τρόπον ἡμεῖς τάς τε δίκας δικάζομεν καὶ τἆλλα τὴν πόλιν διοικοῦμεν, ἤδη φαμὲν τοῦτον ὡμολογηκέναι ἔργῳ ἡμῖν ἃ ἂν ἡμεῖς κελεύωμεν ποιήσειν ταῦτα, καὶ τὸν μὴ πειθόμενον τριχῇ φαμεν ἀδικεῖν, ὅτι τε γεννηταῖς οὖσιν ἡμῖν οὐ πείθεται, καὶ ὅτι τροφεῦσι, καὶ ὅτι ὁμολογήσας ἡμῖν πείσεσθαι οὔτε πείθεται οὔτε πείθει ἡμᾶς, εἰ μὴ καλῶς τι ποιοῦμεν, προτιθέντων ἡμῶν καὶ οὐκ ἀγρίως ἐπιταττόντων ποιεῖν ἃ ἂν κελεύωμεν, ἀλλὰ ἐφιέντων δυοῖν θάτερα, ἢ πείθειν ἡμᾶς ἢ ποιεῖν, τούτων οὐδέτερα ποιεῖ.","[0] But we say that whoever of you stays here, seeing how we administer justice and how we govern the state in other respects, has thereby entered into an agreement with us to do what we command; [1] and we say that he who does not obey does threefold wrong, because he disobeys us who are his parents, because he disobeys us who nurtured him, and because after agreeing to obey us he neither obeys us nor convinces us that we are wrong, though we give him the opportunity and do not roughly order him to do what we command, but when we allow him a choice of two things, either to convince us of error or to do our bidding, he does neither of these things.”","[0] But he who has experience of the manner in which we order justice and administer the State, and still remains, has entered into an implied contract that he will do as we command him. [1] And he who disobeys us is, as we maintain, thrice wrong: [2] first, because in disobeying us he is disobeying his parents; [3] secondly, because we are the authors of his education; [4] thirdly, because he has made an agreement with us that he will duly obey our commands; [5] and he neither obeys them nor convinces us that our commands are wrong; [6] and we do not rudely impose them, but give him the alternative of obeying or convincing us;"
1,"ταύταις δή φαμεν καὶ σέ, ὦ Σώκρατες, ταῖς αἰτίαις ἐνέξεσθαι, εἴπερ ποιήσεις ἃ ἐπινοεῖς, καὶ οὐχ ἥκιστα Ἀθηναίων σέ, ἀλλ᾽ ἐν τοῖς μάλιστα.","[2] and we say that he who does not obey does threefold wrong, because he disobeys us who are his parents, because he disobeys us who nurtured him, and because after agreeing to obey us he neither obeys us nor convinces us that we are wrong, though we give him the opportunity and do not roughly order him to do what we command, but when we allow him a choice of two things, either to convince us of error or to do our bidding, he does neither of these things.”","[7] that is what we offer and he does neither. [8] These are the sort of accusations to which, as we were saying, you, Socrates, will be exposed if you accomplish your intentions; [9] you, above all other Athenians."""


# Stanza taking too long on homer_1866

In [13]:
import stanza
import pandas as pd

from itertools import chain

from functions import (
    load_txt_as_lst, split_txt, flatten_list, write_file)

In [14]:
def run_stanza(text_str, model_):
    '''
    returns sentences as list
    '''
    print("inside run stanza function")
    doc = model_(text_str)
    return [sentence.text for sentence in doc.sentences]

def segment_series(txt_str, lang, model):
    if lang == 'el':
        # split on ;:. for el
        return split_txt(txt_str, lang)
    else:
        # run stanza
        print("running stanza")
        series_sents = run_stanza(txt_str, model)
        # split further on ;:
        series_split = []
        for sent in series_sents:
            series_split.append(split_txt(sent, lang))
        return flatten_list(series_split)

def preprocess_series(txt_lst, lang, model):
    # remove whitespace at beginning and end
    cleaned_str = ""
    # join into str
    for row in txt_lst:
        if row == "":
            continue
        else:
            cleaned_str += row.strip()
    print("cleaned string")
   
    # split text into sentences
    series_split = segment_series(cleaned_str, lang, model)
    
    # save as df and change col name
    series_df = pd.DataFrame(series_split)
    series_df.columns = ['text']
    
    # remove whitespace at beginning and end
    series_df['text'] = series_df['text'].str.strip()

    # drop rows with NaN
    series_df.dropna(how='any', inplace=True)

    # send to list
    series_lst = list(series_df['text'])

    return series_lst

In [15]:
homer_1866 = "/scratch/craig.car/french_trans-dev/homer_1866.txt"
homer_lst = load_txt_as_lst(homer_1866)

In [4]:
stanza_model = stanza.Pipeline(lang="fr", processors='tokenize')

2023-11-15 19:38:11 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-15 19:38:11 WARNING: Language fr package default expects mwt, which has been added
2023-11-15 19:38:11 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |

2023-11-15 19:38:11 INFO: Using device: cpu
2023-11-15 19:38:11 INFO: Loading: tokenize
2023-11-15 19:38:11 INFO: Loading: mwt
2023-11-15 19:38:11 INFO: Done loading processors!


In [16]:
text_sents = preprocess_series(homer_lst, "fr", stanza_model)

cleaned string
running stanza
inside run stanza function


KeyboardInterrupt: 

# check for errors in final sents files

In [33]:
aesch_1863_sents = "/scratch/craig.car/french_trans-dev/aeschylus_1863.sents"
aesch_1863_lst = load_txt_as_lst(aesch_1863_sents)
aesch_1863_df = pd.DataFrame(aesch_1863_lst)
aesch_1863_df

,0
0,"D’un admirable chef-d’œuvre, je n’ai pris, je ..."
1,"Moulé sur un marbre éclatant, radieux, vivant,..."
2,"Mais le modèle est si beau que, même à ses plu..."
3,on a généralement loué leur entreprise.\n
4,"Chez quelques-uns même, ce n’est pas seulement..."
...,...
7469,"je lis, avec Weil :\n"
7470,N’excite pas mon peuple à des discords affreux...
7471,j’adopte la correction que plusieurs critiques...
7472,Et malheur à qui sent de leurs terribles peine...


In [39]:
aesch_dups = pd.DataFrame(aesch_1863_df.duplicated(keep=False))
aesch_dups

,0
0,False
1,False
2,False
3,False
4,False
...,...
7469,True
7470,False
7471,False
7472,False


In [51]:
aesch_dups.index[aesch_dups[0]==True]

Int64Index([ 154,  155,  156,  157,  158,  159,  160,  161,  162,  163,
            ...
            7428, 7429, 7430, 7431, 7432, 7433, 7439, 7468, 7469, 7473],
           dtype='int64', length=7012)

In [46]:
aesch_dups.index[aesch_dups[0]==True].tolist()[:10]

[154, 155, 156, 157, 158, 159, 160, 161, 162, 163]

In [48]:
aesch_dups.index[aesch_dups[0]==True].tolist() == [num for num in range(154, len(aesch_dups.index[aesch_dups[0]==True].tolist()))]
                                                   

False

In [52]:
for num in range(154, len(aesch_dups.index[aesch_dups[0]==True].tolist())):
aesch_dups.index[aesch_dups[0]==True].tolist())

7012

In [56]:
test = [num for num in range(154, len(aesch_dups.index[aesch_dups[0]==True].tolist()))]

In [58]:
len(test)

6858

In [59]:
test[-1]

7011